In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
!pip install langchain
!pip install faiss-cpu
!pip install pyttsx3
!pip install transformers
!pip install torch
!pip install sentence-transformers



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install pymupdf




Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install SpeechRecognition
!pip install pyaudio


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import fitz  # PyMuPDF

def load_pdf(path):
    doc = fitz.open("DSA Full Notes GR-20.pdf")
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = "DSA Full Notes GR-20.pdf"  # Make sure this file is in the same folder as your notebook
raw_text = load_pdf(pdf_path)
print("Text length:", len(raw_text))


Text length: 84972


In [6]:
!pip install sentence-transformers
!pip install faiss-cpu


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define your chunking method
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Each chunk will have ~500 characters
    chunk_overlap=100  # Overlap between chunks (helps for continuity)
)

chunks = text_splitter.split_text(raw_text)
print(f"Total chunks created: {len(chunks)}")


Total chunks created: 213


In [8]:
import ollama
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pyttsx3

# ------------------------------
# 1. Extract text from PDF
# ------------------------------
def extract_pdf_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# ------------------------------
# 2. Split text into chunks
# ------------------------------
def split_into_chunks(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# ------------------------------
# 3. Create FAISS index
# ------------------------------
def create_faiss_index(chunks, model):
    embeddings = model.encode(chunks)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings

# ------------------------------
# 4. Find top N best chunks
# ------------------------------
def find_top_chunks(question, chunks, model, index, top_n=3):
    q_embedding = model.encode([question])
    distances, indices = index.search(np.array(q_embedding), k=top_n)
    return [chunks[i] for i in indices[0]]

# ------------------------------
# 5. Ask TinyLLaMA using Ollama
# ------------------------------
def ask_local_llm(context, question):
    prompt = f"""You are a helpful AI tutor. Based only on the textbook content below, answer the question clearly and accurately.

Text:
{context}

Question:
{question}

Answer:"""
    response = ollama.chat(
        model='tinyllama',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response['message']['content']

# ------------------------------
# 6. Speak the answer (TTS)
# ------------------------------
def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# ------------------------------
# MAIN SCRIPT (Interactive Chat with Voice)
# ------------------------------
if __name__ == "__main__":
    pdf_path = r"DSA Full Notes GR-20.pdf"  # <-- Your PDF file path

    print("📖 Extracting PDF text...")
    full_text = extract_pdf_text(pdf_path)
    if not full_text.strip():
        print("❌ Could not extract text from the PDF.")
        exit()

    print("🔄 Splitting into chunks...")
    chunks = split_into_chunks(full_text)
    print(f"✅ PDF split into {len(chunks)} chunks.")

    print("🔍 Creating FAISS index...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    index, embeddings = create_faiss_index(chunks, model)
    print("✅ Ready! Ask any question (type 'exit' to quit).")

    # Interactive Loop
    while True:
        question = input("\n❓ Your Question: ")
        if question.lower() in ["exit", "quit", "bye"]:
            print("👋 Exiting AI Tutor. Goodbye!")
            break

        print("🔍 Searching PDF for relevant content...")
        top_chunks = find_top_chunks(question, chunks, model, index, top_n=3)
        combined_context = "\n\n".join(top_chunks)

        print("🤖 Generating answer...\n")
        answer = ask_local_llm(combined_context, question)

        print(f"💬 AI Tutor Answer:\n{answer}\n")

        # Speak answer
        print("🔊 Speaking answer...")
        speak_text(answer)


📖 Extracting PDF text...
🔄 Splitting into chunks...
✅ PDF split into 26 chunks.
🔍 Creating FAISS index...
✅ Ready! Ask any question (type 'exit' to quit).
🔍 Searching PDF for relevant content...
🤖 Generating answer...

💬 AI Tutor Answer:
Data science is the field of study that involves collecting, analyzing, and interpreting data. It involves a combination of mathematics, statistics, computer science, and engineering to gain insights from large amounts of data. Data science is increasingly becoming an essential component of modern businesses and industries, as it enables them to make better decisions, improve their products or services, and optimize their processes. Some key areas of application for data science include:

1. Predictive analytics: This is a discipline that uses data to make predictions about future outcomes based on past data. Predictive analytics allows businesses to identify patterns in customer behavior or sales trends and use that information to make informed decisi

In [9]:
import ollama
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pyttsx3
import speech_recognition as sr

# ------------------------------
# 1. Extract text from PDF
# ------------------------------
def extract_pdf_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# ------------------------------
# 2. Split text into chunks
# ------------------------------
def split_into_chunks(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# ------------------------------
# 3. Create FAISS index
# ------------------------------
def create_faiss_index(chunks, model):
    embeddings = model.encode(chunks)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings

# ------------------------------
# 4. Find top N best chunks
# ------------------------------
def find_top_chunks(question, chunks, model, index, top_n=3):
    q_embedding = model.encode([question])
    distances, indices = index.search(np.array(q_embedding), k=top_n)
    return [chunks[i] for i in indices[0]]

# ------------------------------
# 5. Ask TinyLLaMA using Ollama
# ------------------------------
def ask_local_llm(context, question):
    prompt = f"""You are a helpful AI tutor. Based only on the textbook content below, answer the question clearly and accurately.

Text:
{context}

Question:
{question}

Answer:"""
    response = ollama.chat(
        model='tinyllama',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response['message']['content']

# ------------------------------
# 6. Speak the answer (TTS)
# ------------------------------
def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# ------------------------------
# 7. Listen for voice question
# ------------------------------
def listen_question():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Listening... Please speak your question.")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        question = recognizer.recognize_google(audio)
        print(f"❓ You asked: {question}")
        return question
    except sr.UnknownValueError:
        print("❌ Sorry, I couldn't understand. Please try again.")
        return None
    except sr.RequestError:
        print("❌ Voice recognition service is unavailable.")
        return None

# ------------------------------
# MAIN SCRIPT
# ------------------------------
if __name__ == "__main__":
    pdf_path = r"DSA Full Notes GR-20.pdf"  # <-- Your PDF file

    print("📖 Extracting PDF text...")
    full_text = extract_pdf_text(pdf_path)
    if not full_text.strip():
        print("❌ Could not extract text from the PDF.")
        exit()

    print("🔄 Splitting into chunks...")
    chunks = split_into_chunks(full_text)
    print(f"✅ PDF split into {len(chunks)} chunks.")

    print("🔍 Creating FAISS index...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    index, embeddings = create_faiss_index(chunks, model)
    print("✅ Ready! Say 'exit' to quit.")

    while True:
        question = listen_question()
        if not question:
            continue
        if question.lower() in ["exit", "quit", "bye"]:
            print("👋 Exiting AI Tutor. Goodbye!")
            speak_text("Goodbye! Exiting AI Tutor.")
            break

        print("🔍 Searching PDF for relevant content...")
        top_chunks = find_top_chunks(question, chunks, model, index, top_n=3)
        combined_context = "\n\n".join(top_chunks)

        print("🤖 Generating answer...\n")
        answer = ask_local_llm(combined_context, question)

        print(f"💬 AI Tutor Answer:\n{answer}\n")
        speak_text(answer)




📖 Extracting PDF text...
🔄 Splitting into chunks...
✅ PDF split into 26 chunks.
🔍 Creating FAISS index...
✅ Ready! Say 'exit' to quit.
🎤 Listening... Please speak your question.
❌ Sorry, I couldn't understand. Please try again.
🎤 Listening... Please speak your question.
❌ Sorry, I couldn't understand. Please try again.
🎤 Listening... Please speak your question.
❓ You asked: machine learning
🔍 Searching PDF for relevant content...
🤖 Generating answer...

💬 AI Tutor Answer:
Sure! Machine learning is a powerful technique for predicting and generating new data based on historical data. One of the most popular machine learning models, which are used to build predictive models, is called support vector machines (SVMs). SVMs work by maximizing the margin between two classes, which is a measure of how different their features (attributes) are compared to each other.

The advantage of using SVMs for predicting new data is that they can handle both high-dimensional and noisy data with ease. The 

In [10]:

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# ------------------------------
# 1. Test Questions & Expected Answers
# ------------------------------
qa_pairs = [
    ("What is predictive analysis?", "Predictive analysis uses historical data and statistical models to predict future outcomes."),
    ("What is regression?", "Regression is a supervised learning method used to predict continuous values."),
    ("What is classification?", "Classification is a supervised learning task that assigns data into predefined labels."),
    ("What are the steps of data preprocessing?", "Data preprocessing involves data cleaning, normalization, transformation, feature selection, and splitting."),
    ("What is clustering?", "Clustering is an unsupervised learning technique that groups similar data points."),
]

# ------------------------------
# 2. Load Similarity Model
# ------------------------------
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to compute semantic similarity
def get_similarity(ans1, ans2):
    emb1 = similarity_model.encode([ans1])
    emb2 = similarity_model.encode([ans2])
    return cosine_similarity(emb1, emb2)[0][0]

# ------------------------------
# 3. Accuracy Test
# ------------------------------
correct = 0
print("\n🔍 Starting accuracy test...\n")

for i, (question, expected) in enumerate(qa_pairs, 1):
    # Use your existing chunk search and LLM answering functions
    top_chunks = find_top_chunks(question, chunks, model, index, top_n=2)
    context = "\n".join(top_chunks)
    ai_answer = ask_local_llm(context, question)
    
    # Calculate similarity
    sim = get_similarity(ai_answer, expected)
    print(f"Q{i}: {question}")
    print(f"AI Answer: {ai_answer}")
    print(f"Expected: {expected}")
    print(f"Similarity Score: {sim:.2f}\n")
    
    if sim > 0.7:
        correct += 1

accuracy = (correct / len(qa_pairs)) * 100
print(f"✅ Final Accuracy of AI Tutor: {accuracy:.2f}%")



🔍 Starting accuracy test...

Q1: What is predictive analysis?
AI Answer: Predictive Analytics refers to the process of using advanced algorithms and statistical techniques to predict future outcomes based on historical data and past patterns. This can be applied in a variety of domains such as healthcare, finance, manufacturing, etc., with the aim of improving decision-making, reducing costs, and enhancing profitability for organizations. Predictive analytics tools utilize machine learning algorithms and data science techniques to analyze historical data and make predictions about future outcomes. By forecasting and predicting, predictive analytics enables organizations to take proactive measures to anticipate and mitigate risks or opportunities, improve performance, and optimize decision-making processes.
Expected: Predictive analysis uses historical data and statistical models to predict future outcomes.
Similarity Score: 0.77

Q2: What is regression?
AI Answer: Regresión (literally